In [1]:
import pyxacc as xacc
import pyxaccvqe as vqe
from openfermion.hamiltonians import MolecularData
from openfermionpsi4 import run_psi4

In [2]:
geometry = [('H', (0,0,0)),('H',(0,0,.7474))]
basis = 'sto-3g'
mdata = MolecularData(geometry,basis,1)
molecule = run_psi4(mdata, run_scf=1, run_fci=1)
print(molecule.fci_energy)

-1.1371757101731694


In [5]:
%ls /root/.local/lib/python3.6/site-packages/psi4/bin


psi4*  xc-info*  xc-threshold*
